In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function
import os

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import KeyedVectors
from scipy.spatial.distance import cosine
from sklearn.metrics import euclidean_distances
from pyemd import emd

import pandas as pd
from nltk.tokenize import word_tokenize
import nltk
import string

In [ ]:
def check_value(w):
    if(vocab_dict.has_key(w)):
        return(vocab_dict[w])
    return 0

def lexicon_rate(lexicon, comment):
    vect = CountVectorizer(token_pattern=pattern, strip_accents=None).fit([lexicon, comment])
    v_1, v_2 = vect.transform([lexicon, comment])
    v_1 = v_1.toarray().ravel()
    v_2 = v_2.toarray().ravel()
    W_ = W[[check_value(w) for w in vect.get_feature_names()]]
    D_ = euclidean_distances(W_)
    v_1 = v_1.astype(np.double)
    v_2 = v_2.astype(np.double)
    v_1 /= v_1.sum()
    v_2 /= v_2.sum()
    D_ = D_.astype(np.double)
    D_ /= D_.max()
    lex=emd(v_1, v_2, D_)
    return(lex)

In [ ]:
wv = KeyedVectors.load_word2vec_format('../../data/wikipedia_embedding/data/wiki_vectors_format_without_stopwords.bin', binary=False, unicode_errors="ignore")
wv.init_sims()

In [ ]:
pattern = "(?u)\\b[\\w-]+\\b"

fp = np.memmap("../../data/wikipedia_embedding/data/embed.dat", dtype=np.double, mode='w+', shape=wv.syn0norm.shape)
fp[:] = wv.syn0norm[:]
with open("../../data/wikipedia_embedding/data/embed.vocab", "w") as f:
    for _, w in sorted((voc.index, word) for word, voc in wv.vocab.items()):
        print(w.encode('utf-8'), file=f)

vocab_len = len(wv.vocab)
del fp, wv

W = np.memmap("../../data/wikipedia_embedding/data/embed.dat", dtype=np.double, mode="r", shape=(vocab_len, 300))

with open("../../data/wikipedia_embedding/data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())
vocab_dict={w:k for k, w in enumerate(vocab_list)}

In [ ]:
def open_file(name):
    with open(name, "r") as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    return content

In [ ]:
argumentacao = "a_ponto ao_menos apenas ate ate_mesmo incluindo inclusive mesmo nao_mais_que nem_mesmo no_minimo o_unico a_unica pelo_menos quando_menos quando_muito sequer so somente a_par_disso ademais afinal ainda alem alias como e e_nao em_suma enfim mas_tambem muito_menos nao_so nem ou_mesmo por_sinal tambem tampouco assim com_isso como_consequencia consequentemente de_modo_que deste_modo em_decorrencia entao logicamente logo nesse_sentido pois por_causa por_conseguinte por_essa_razao por_isso portanto sendo_assim ou ou_entao ou_mesmo nem como_se de_um_lado por_outro_lado mais_que menos_que nao_so tanto quanto tao como desde_que do_contrario em_lugar em_vez enquanto no_caso quando se se_acaso senao de_certa_forma desse_modo em_funcao enquanto isso_e ja_que na_medida_que nessa_direcao no_intuito no_mesmo_sentido ou_seja pois porque que uma_vez_que tanto_que visto_que ainda_que ao_contrario apesar_de contrariamente contudo embora entretanto fora_isso mas mesmo_que nao_obstante nao_fosse_isso no_entanto para_tanto pelo_contrario por_sua_vez porem posto_que todavia"
modalizacao = "achar aconselhar acreditar aparente basico bastar certo claro conveniente crer dever dificil duvida efetivo esperar evidente exato facultativo falar fato fundamental imaginar importante indubitavel inegavel justo limitar logico natural necessario negar obrigatorio obvio parecer pensar poder possivel precisar predominar presumir procurar provavel puder real recomendar seguro supor talvez tem tendo ter tinha tive verdade decidir"
valoracao = "absoluto algum alto amplo aproximado bastante bem bom categorico cerca completo comum consideravel constante definitivo demais elevado enorme escasso especial estrito eventual exagero excelente excessivo exclusivo expresso extremo feliz franco franqueza frequente generalizado geral grande imenso incrivel lamentavel leve maioria mais mal melhor menos mero minimo minoria muito normal ocasional otimo particular pena pequeno pesar pior pleno pobre pouco pouquissimo praticamente prazer preciso preferir principal quase raro razoavel relativo rico rigor sempre significativo simples tanto tao tipico total tremenda usual valer"
sentimento = "abalar abater abominar aborrecer acalmar acovardar admirar adorar afligir agitar alarmar alegrar alucinar amar ambicionar amedrontar amolar animar apavorar apaziguar apoquentar aporrinhar apreciar aquietar arrepender assombrar assustar atazanar atemorizar aterrorizar aticar atordoar atormentar aturdir azucrinar chatear chocar cobicar comover confortar confundir consolar constranger contemplar contentar contrariar conturbar curtir debilitar decepcionar depreciar deprimir desapontar descontentar descontrolar desejar desencantar desencorajar desesperar desestimular desfrutar desgostar desiludir desinteressar deslumbrar desorientar desprezar detestar distrair emocionar empolgar enamorar encantar encorajar endividar enervar enfeiticar enfurecer enganar enraivecer entediar entreter entristecer entusiasmar envergonhar escandalizar espantar estimar estimular estranhar exaltar exasperar excitar execrar fascinar frustar gostar gozar grilar hostilizar idolatrar iludir importunar impressionar incomodar indignar inibir inquietar intimidar intrigar irar irritar lamentar lastimar louvar magoar maravilhar melindrar menosprezar odiar ofender pasmar perdoar preocupar prezar querer recalcar recear reconfortar rejeitar repelir reprimir repudiar respeitar reverenciar revoltar seduzir sensibilizar serenar simpatizar sossegar subestimar sublimar superestimar surpreender temer tolerar tranquilizar transtornar traumatizar venerar" 
pressuposicao = "adivinhar admitir agora aguentar ainda antes atentar atual aturar comecar compreender conseguir constatar continuar corrigir deixar demonstrar descobrir desculpar desde desvendar detectar entender enxergar esclarecer escutar esquecer gabar ignorar iniciar interromper ja lembrar momento notar observar olhar ouvir parar perceber perder pressentir prever reconhecer recordar reparar retirar revelar saber sentir tolerar tratar ver verificar"

df = pd.read_csv("../../data/wikipedia_embedding/data/csv_fake_95_informat.csv")

content = [df['text'][i].strip() for i in range(len(df)) if df['label'][i] == 'fake']

In [ ]:
def loadStopWordsPT(filename):
    lines = [line.rstrip('\n').strip() for line in open(filename)]
    return lines

raw_stop_words = loadStopWordsPT('stop_words_noAccents_pt_plusBACKUP.txt')
        
def clean_stopwords(text, stop_words_list):
    list_words = text.split()
    list_clean_text = []
    for word in list_words:
        if word not in stop_words_list:
            list_clean_text.append(word)
    return " ".join(list_clean_text)

In [ ]:
argumentacao = clean_stopwords(argumentacao, raw_stop_words)
modalizacao = clean_stopwords(modalizacao, raw_stop_words)
valoracao = clean_stopwords(valoracao, raw_stop_words)
sentimento = clean_stopwords(sentimento, raw_stop_words)
pressuposicao = clean_stopwords(pressuposicao, raw_stop_words)

for i in range(len(content)):
    content[i] = clean_stopwords(content[i], raw_stop_words)
    content[i] = unicode(content[i], "utf-8")
    sent_text = nltk.sent_tokenize(content[i])
    content[i] = [sentence.encode("utf-8") for sentence in sent_text]

In [ ]:
def restore_text(list_sentences):
    original_text = ''
    for sentence in list_sentences:
        original_text += sentence
    return original_text

In [ ]:
def remove_dirty_sentences(list_sentences):
    clean_sentences = []
    for sentence in list_sentences:
        if(len(sentence.split()) > 2):
            clean_sentences.append(sentence)
    return clean_sentences

In [ ]:
for i in range(len(content)):
    content[i] = remove_dirty_sentences(content[i])

content = [lista for lista in content if len(lista) > 0]

In [ ]:
lexicons_rates = list()
iter_count = 0
for sentences in content:
    arg_avg = 0
    sen_avg = 0
    mod_avg = 0
    val_avg = 0
    pre_avg = 0

    for sentence in sentences:
        arg_avg += lexicon_rate(argumentacao, sentence)
        mod_avg += lexicon_rate(modalizacao, sentence)
        val_avg += lexicon_rate(valoracao, sentence)
        sen_avg += lexicon_rate(sentimento, sentence)
        pre_avg += lexicon_rate(pressuposicao, sentence)
    
    arg_avg = arg_avg / float(len(sentences))
    sen_avg = sen_avg / float(len(sentences))
    mod_avg = mod_avg / float(len(sentences))
    val_avg = val_avg / float(len(sentences))
    pre_avg = pre_avg / float(len(sentences))
    
    rates = list([restore_text(sentences),'fake',arg_avg, sen_avg, val_avg, mod_avg, pre_avg])
    lexicons_rates.append(rates)

In [ ]:
df = pd.DataFrame(lexicons_rates, columns=['Comment','class','arg','sen','val','mod','pre'])

In [ ]:
df.to_csv("../../data/wikipedia_embedding/body/wmd_distances_FAKE_95.csv", index=False)